In [1]:
import numpy as np
import pandas as pd

# Feature Engineering

In [2]:
auto = pd.read_csv("./clean-auto-mpg.csv")
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1


Right off the bat it can be seen that `origin` is not a numerical field. It needs to be changed to one-hot.

In [3]:
auto["USA"] = (auto["origin"] == 1)*1.
auto["Europe"] = (auto["origin"] == 2)*1.
auto["Japan"] = (auto["origin"] == 3)*1.
auto.tail(5)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,USA,Europe,Japan
387,27.0,4,140.0,86,2790,15.6,82,1,1.0,0.0,0.0
388,44.0,4,97.0,52,2130,24.6,82,2,0.0,1.0,0.0
389,32.0,4,135.0,84,2295,11.6,82,1,1.0,0.0,0.0
390,28.0,4,120.0,79,2625,18.6,82,1,1.0,0.0,0.0
391,31.0,4,119.0,82,2720,19.4,82,1,1.0,0.0,0.0


## Train, Val, and Test splits

In [4]:
auto_train = auto.sample(frac=0.8)
auto_ = auto.drop(auto_train.index)
auto_val = auto_.sample(frac=0.5)
auto_test = auto_.drop(auto_val.index)
print(len(auto_train), len(auto_val), len(auto_test))

314 39 39


## Normalize
The following columns which will be used as inputs, have very widely differing scales. They need to be normalized so that their means are 0 and std devs are 1.
  * cylinders
  * displacement
  * horsepower
  * weight
  * acceleration

For all three datasets, use the train set's statistics to normalize.

In [6]:
train_stats = auto_train.describe()
train_stats

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.296497,5.477707,195.136943,104.608280,2982.455414,15.642675,75.859873,1.566879,0.627389,0.178344,0.194268
std,7.799190,1.711028,103.772070,37.655188,819.011447,2.850014,3.740733,0.797551,0.484272,0.383413,0.396267
min,9.000000,3.000000,70.000000,46.000000,1613.000000,8.000000,70.000000,1.000000,0.000000,0.000000,0.000000
25%,17.125000,4.000000,107.000000,78.000000,2254.250000,13.925000,73.000000,1.000000,0.000000,0.000000,0.000000
50%,22.000000,4.000000,151.000000,95.000000,2832.500000,15.500000,76.000000,1.000000,1.000000,0.000000,0.000000
75%,28.000000,8.000000,293.250000,125.000000,3618.250000,17.275000,79.000000,2.000000,1.000000,0.000000,0.000000
max,46.600000,8.000000,455.000000,225.000000,4955.000000,24.800000,82.000000,3.000000,1.000000,1.000000,1.000000


In [7]:
cols_to_norm = ["cylinders", "displacement", "horsepower", "weight", "acceleration"]
for col in cols_to_norm:
    mean = train_stats[col].loc["mean"]
    std = train_stats[col].loc["std"]
    auto_train[col] = (auto_train[col] - mean) / std
    auto_val[col] = (auto_val[col] - mean) / std
    auto_test[col] = (auto_test[col] - mean) / std

Now the mean of these columns should be 0 and their std dev should be 1. Lets verify.

In [9]:
auto_train.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,USA,Europe,Japan
count,314.000000,3.140000e+02,3.140000e+02,3.140000e+02,3.140000e+02,3.140000e+02,314.000000,314.000000,314.000000,314.000000,314.000000
mean,23.296497,1.470869e-16,-1.343582e-16,-5.657187e-18,-2.828594e-18,5.968333e-16,75.859873,1.566879,0.627389,0.178344,0.194268
std,7.799190,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.740733,0.797551,0.484272,0.383413,0.396267
min,9.000000,-1.448081e+00,-1.205883e+00,-1.556446e+00,-1.672083e+00,-2.681627e+00,70.000000,1.000000,0.000000,0.000000,0.000000
25%,17.125000,-8.636370e-01,-8.493320e-01,-7.066299e-01,-8.891273e-01,-6.026900e-01,73.000000,1.000000,0.000000,0.000000,0.000000
50%,22.000000,-8.636370e-01,-4.253258e-01,-2.551648e-01,-1.830932e-01,-5.006121e-02,76.000000,1.000000,1.000000,0.000000,0.000000
75%,28.000000,1.474139e+00,9.454669e-01,5.415381e-01,7.762951e-01,5.727427e-01,79.000000,2.000000,1.000000,0.000000,0.000000
max,46.600000,1.474139e+00,2.504171e+00,3.197215e+00,2.408446e+00,3.213080e+00,82.000000,3.000000,1.000000,1.000000,1.000000


# Conclusion
The original *clean-auto-mpg.csv* has been split into train, val, and test sets and the data within them normalized appropriately. Lets write these split CSVs.

In [10]:
auto_train.to_csv("./train-auto-mpg.csv", index=False)
auto_val.to_csv("./val-auto-mpg.csv", index=False)
auto_test.to_csv("./test-auto-mpg.csv", index=False)